In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import json, os, time, re
from pprint import pprint
from urllib import parse
import requests 

# 隨機取得 User-Agent
from fake_useragent import UserAgent
ua = UserAgent(cache=True) # cache=True 表示從已經儲存的列表中提取

my_options = webdriver.ChromeOptions()
my_options.add_argument('--start-maximized')
my_options.add_argument('--incognito')
my_options.add_argument('--disable-popup-blocking')
my_options.add_argument(f'--user-agent={ua.random}')

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)

listData = []

'''小說的網址'''
url = 'https://www.gutenberg.org/browse/languages/zh'

# 沒有放置 txt 檔的資料夾，就建立起來
folderPath = 'gutenburg'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

In [ ]:
def getMainLink():
    driver.get(url)
    
    a_elms = driver.find_elements(
        By.CSS_SELECTOR, 
        'li.pgdbetext > a')
    
    regex = r"\/([a-z]+)\/(\d+)$"
    for a in a_elms:
        link = parse.unquote(a.get_attribute('href'))
        match = re.search(regex, link)
        newLink = link.replace(match[1], "files")
        newLink = newLink + "/" + match[2] + "-0.txt"
        listData.append({
            'title': a.get_attribute('innerText'),
            'link': newLink
        })
    #print(listData)
    
def saveJson():
    with open(f"{folderPath}/gutenburg.json", "w", encoding="utf-8") as file:
        file.write(json.dumps(listData, ensure_ascii=False))
        
def writeTxt():
    listContent = []
    
    with open(f"{folderPath}/gutenburg.json", "r", encoding='utf-8') as file:
        strJson = file.read()
        
    listResult = json.loads(strJson)
    for i in range(len(listResult)):
        driver.get(listResult[i]['link'])
        r = requests.get(listResult[i]['link'])
        if r.status_code != 200:
            continue
        else:
            div = driver.find_element(By.CSS_SELECTOR, "body")

            strContent = div.get_attribute('innerText')
            strContent = re.sub(r" |\r|\n|　|\s", '', strContent)

            fileName = f"{listResult[i]['title']}.txt"
            with open(f"{folderPath}/{fileName}", "w", encoding="utf-8") as file:
                file.write(strContent)

            listContent.append(strContent)

    with open(f"{folderPath}/train.json", "w", encoding="utf-8") as file:
            file.write( json.dumps(listContent, ensure_ascii=False) )
                                  
def close():
    driver.quit()   

In [ ]:
if __name__ == '__main__':
    getMainLink()
    saveJson()
    writeTxt()
    close()